# Plan your trip with Kayak



Contents
--------
1. [Fetching data from the database](#fetch)
2. [Application](#application)

## <a name="fetch"></a>Fetching data from the database



In [3]:
import pandas as pd
from sqlalchemy import create_engine, URL, inspect
from sqlalchemy import text
from sqlalchemy.orm import Session

In [4]:
## get credentials
with open('./neondb_access_keys.key', 'rt', encoding='utf-8') as f:
    PGHOST = f.readline().split("'")[1]
    PGDATABASE = f.readline().split("'")[1]
    PGUSER = f.readline().split("'")[1]
    PGPASSWORD = f.readline().split("'")[1]

url = URL.create(
    "postgresql+psycopg",
    username=PGUSER,
    password=PGPASSWORD,
    host=PGHOST,
    database=PGDATABASE,
)

## setup SQL engine
engine = create_engine(url, echo=False)
# inspector = inspect(engine)

In [5]:
locations = pd.read_sql(text("SELECT * FROM locations"), con=engine)
# weather_indicators = pd.read_sql(text("SELECT * FROM weather_indicators"), con=engine)
# hotels = pd.read_sql(text("SELECT * FROM hotels"), con=engine)

## <a name="application"></a>Application